In [1]:
! pip install -qU torch transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.1 MB/s eta 0:00:00


In [3]:
pip install -qU flash_attn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.9 MB/s eta 0:00:00


In [ ]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM, AutoProcessor
import torch

In [ ]:
model_id = "microsoft/Phi-3-vision-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code =True, torch_dtype='auto',device_map='auto')
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
url = "https://assets-c4akfrf5b4d3f4b7.z01.azurefd.net/assets/2024/04/BMDataViz_661fb89f3845e.png"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
messages = [
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"},
    {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."},
    {"role": "user", "content": "Provide insightful questions to spark discussion."}
]

In [ ]:
prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
inputs = processor(prompt,[image],return_tensors='pt').to(model.device)

In [ ]:
inputs

In [ ]:
args={
    "max_new_tokens": 1000,
    "temperature": 0.5,
    "do_sample": False,
}

In [ ]:
output = model.generate(**inputs,eos_token_id=processor.tokenizer.eos_token_id,**args)